In [59]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time
import os

In [60]:
sampleSize = 50000
sample_rate = 16000
quantization_channels = 256
# dilations=[2**i for i in range(8)]*20
# "residualDim=32
dilations = [2 ** i for i in range(9)] * 5
residualDim = 32
skipDim = 512
filterSize = 3
initfilter=3
pad = np.sum(dilations)
shapeoftest = 190500
lossrecord = []
resumefile='classifyregress'
continueTrain=True
receptive_field = (sum(dilations) + 1)
pad

2555

In [61]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [62]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
device = 'cpu'
#torch.set_default_tensor_type('torch.cuda.FloatTensor')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [63]:
def mu_law_encode(audio, quantization_channels=quantization_channels,forX=False):
    '''Quantizes waveform amplitudes.'''
    mu = (quantization_channels - 1)*1.0
    # Perform mu-law companding transformation (ITU-T, 1988).
    # Minimum operation is here to deal with rare large amplitudes caused
    # by resampling.
    safe_audio_abs = np.minimum(np.abs(audio), 1.0)
    magnitude = np.log1p(mu * safe_audio_abs) / np.log1p(mu)
    signal = np.sign(audio) * magnitude
    # Quantize signal to the specified number of levels.
    if(forX):return signal
    return ((signal + 1) / 2 * mu + 0.5).astype(int)
def mu_law_decode(output, quantization_channels=quantization_channels):
    '''Recovers waveform from quantized values.'''
    mu = quantization_channels - 1
    # Map values back to [-1, 1].
    signal = 2 * ((output*1.0) / mu) - 1
    # Perform inverse of mu-law transformation.
    magnitude = (1 / mu) * ((1 + mu)**np.abs(signal) - 1)
    return np.sign(signal) * magnitude
def cateToSignal(output, quantization_channels=quantization_channels):
    mu = quantization_channels - 1
    # Map values back to [-1, 1].
    signal = 2 * ((output*1.0) / mu) - 1
    return signal

In [64]:
def readAudio(name):
    audio0, samplerate = sf.read(name, dtype='float32')
    return librosa.resample(audio0.T, samplerate, sample_rate).reshape(-1)
p=['./vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav',
   './vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav','./vsCorpus/pred_mix.wav']
xtrain,ytrain,xval,yval,xtest=readAudio(p[0]),readAudio(p[1]),readAudio(p[2]),readAudio(p[3]),readAudio(p[4])
assert((xtrain==xval).all())
assert((ytrain==yval).all())
assert((xtrain != ytrain).any())

In [65]:
ytrain,yval=mu_law_encode(ytrain),mu_law_encode(yval)
xytrain,xyval=mu_law_encode(ytrain,forX=True),mu_law_encode(yval,forX=True)
xtrain,xval,xtest=mu_law_encode(xtrain,forX=True),mu_law_encode(xval,forX=True),mu_law_encode(xtest,forX=True)

In [66]:
xmean,xstd = xtrain.mean(),xtrain.std()
xtrain=(xtrain-xmean)/xstd
xval=(xval-xmean)/xstd
xtest=(xtest-xmean)/xstd

In [67]:
xtrain=np.pad(xtrain, (receptive_field, receptive_field), 'constant')
xval=np.pad(xval, (receptive_field, receptive_field), 'constant')
xtest=np.pad(xtest, (receptive_field, receptive_field), 'constant')
yval=np.pad(yval, (receptive_field, receptive_field), 'constant')
ytrain=np.pad(ytrain, (receptive_field, receptive_field), 'constant')
xyval=np.pad(xyval, (receptive_field, receptive_field), 'constant')
xytrain=np.pad(xytrain, (receptive_field, receptive_field), 'constant')

In [68]:
xtrain,xval,xtest=xtrain.reshape(1,1,-1),xval.reshape(1,1,-1),xtest.reshape(1,1,-1)
ytrain,yval=ytrain.reshape(1,-1),yval.reshape(1,-1)
xytrain,xyval=ytrain.reshape(1,1,-1),yval.reshape(1,1,-1)

In [69]:
#xtrain = np.concatenate((xtrain,xytrain.reshape(xtrain.shape)),axis=1)
#xval = np.concatenate((xval,xyval.reshape(xval.shape)),axis=1)
#xtest = np.concatenate((xtest,np.zeros_like(xtest)),axis=1)

In [70]:
xtrain,ytrain,xval,yval,xtest = torch.from_numpy(xtrain).type(torch.float32),\
                                torch.from_numpy(ytrain).type(torch.LongTensor),\
                                torch.from_numpy(xval).type(torch.float32),\
                                torch.from_numpy(yval).type(torch.LongTensor),\
                                torch.from_numpy(xtest).type(torch.float32)
xytrain,xyval=torch.from_numpy(xytrain).type(torch.float32),torch.from_numpy(xyval).type(torch.float32),

In [71]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        sd,qd,rd = skipDim,quantization_channels,residualDim
        self.xcausal = nn.Conv1d(in_channels=1,out_channels=rd,kernel_size=initfilter,padding=1)
        self.ycausal = nn.Conv1d(in_channels=1,out_channels=rd,kernel_size=initfilter,padding=1)
        self.ly=dict()
        self.lx=dict()
        for i, d in enumerate(dilations):
            self.lx['tanh'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,dilation=d)
            self.lx['sigmoid'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,dilation=d)
            self.lx['skip'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=sd,kernel_size=1)
            self.lx['dense'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=1)
            
            self.ly['tanh'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=2,dilation=d)
            self.ly['sigmoid'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=2,dilation=d)
            self.ly['skip'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=sd,kernel_size=1)
            self.ly['dense'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=1)
        self.post1 = nn.Conv1d(in_channels=sd,out_channels=sd,kernel_size=1,padding=0)
        self.post2 = nn.Conv1d(in_channels=sd,out_channels=qd,kernel_size=1,padding=0)
        self.tanh,self.sigmoid = nn.Tanh(),nn.Sigmoid()

    def forward(self, x, y):
        finallen = x.shape[-1]-2*receptive_field
        x = self.xcausal(x)
        y = self.ycausal(y)
        skipx = torch.zeros([1,skipDim,finallen],dtype=torch.float32,device=device)
        skipy = torch.zeros([1,skipDim,finallen],dtype=torch.float32,device=device)
        accumulate=1
        for i, dilation in enumerate(dilations):
            accumulate+=dilations
            yinput = y.clone()[:,:,:-dilation]
            y1 = self.tanh(self.ly['tanh'+str(i)](y))
            y2 = self.sigmoid(self.ly['sigmoid'+str(i)](y))
            y=y1*y2
            skipy += (self.ly['skip'+str(i)](y)).\
                narrow(2,int(receptive_field-accumulate),int(finallen))
            
            xinput = x.clone()[:,:,dilation:-dilation]
            x1 = self.tanh(self.lx['tanh'+str(i)](x))
            x2 = self.sigmoid(self.lx['sigmoid'+str(i)](x))
            x = x1*x2
            cutlen = (x.shape[-1] - finallen)//2
            skipx += (self.lx['skip'+str(i)](x)).narrow(2,int(cutlen),int(finallen))
            x = self.lx['dense'+str(i)](x)
            x += xinput
        x = self.post2(F.relu(self.post1(F.relu(torch.cat((skipx,skipy), dim=1)))))
        return x

model = Net().cuda()
criterion = nn.CrossEntropyLoss().cuda()
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
#optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-5)*
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9, weight_decay=1e-5)
#scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
#scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)

In [72]:
if continueTrain:
    if os.path.isfile(resumefile):
        print("=> loading checkpoint '{}'".format(resumefile))
        checkpoint = torch.load(resumefile)
        start_epoch = checkpoint['epoch']
        #best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        #optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(resumefile, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(resumefile))

=> no checkpoint found at 'classifyregress'


In [73]:
def val():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        # data, target = xval.to(device), yval.to(device)
        data, target = xtrain[:, :, 0:2 * receptive_field + shapeoftest].to(device),\
            ytrain[:, receptive_field:shapeoftest + receptive_field].to(device)
        output = model(data)
        # print(output.shape)
        # print(output[:,:,:10])
        pred = output.max(1, keepdim=True)[1]
        correct = pred.eq(target.view_as(pred)).sum().item() / pred.shape[-1]
        val_loss = criterion(output, target).item()
    print(correct, 'accurate')
    print('\nval set:loss{:.4f}:, ({:.3f} sec/step)\n'.format(val_loss, time.time() - startval_time))


def test():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        output = model(xtest.to(device))
        pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
        ans = mu_law_decode(pred)
        sf.write('./vsCorpus/resultxte.wav', ans, sample_rate)

        # output = model(xtrain[:,:,:sampleSize].to(device))
        output = model(xtrain[:, :, 0:2 * receptive_field + shapeoftest].to(device))
        pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
        ans = mu_law_decode(pred)
        sf.write('./vsCorpus/resultxtr.wav', ans, sample_rate)
        print('stored done\n')


def train(epoch):
    model.train()
    # idx = np.arange(xtrain.shape[-1] - 2 * sampleSize,1000)
    # 176000
    idx = np.arange(receptive_field, xtrain.shape[-1] - receptive_field - sampleSize - 1, 16000)
    np.random.shuffle(idx)
    for i, ind in enumerate(idx):
        start_time = time.time()
        datax = xtrain[:,:,ind - receptive_field:ind + sampleSize + receptive_field].to(device)
        datay = xytrain[:,:,ind - receptive_field:ind + sampleSize + receptive_field].to(device)
        target = ytrain[:,ind:ind + sampleSize].to(device)
            
        output = model(datax,datay)
        # print(output.shape)
        loss = criterion(output, target)
        lossrecord.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)] Loss:{:.6f}: , ({:.3f} sec/step)'.format(
            epoch, i, len(idx), 100. * i / len(idx), loss.item(), time.time() - start_time))
        if i % 100 == 0:
            #val()
            #test()
            state={'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()}
            torch.save(state, resumefile)



In [74]:
for epoch in range(100000):
    train(epoch)
    #test()
    #scheduler.step()

RuntimeError: Expected object of type torch.FloatTensor but found type torch.cuda.FloatTensor for argument #2 'weight'

In [20]:
#torch.save(model, 'loss3.2step24h_repeat5*2**10resu96sample50000')

/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
